# Creating a Cognite Function directly from a notebook

In this notebook we showcase how we can use the Cognite Python SDK to create a Cognite Function directly from a notebook cell. We create a Cognite Function that takes as input the numerical ID of a Cognite Asset and then prints and returns the corresponding asset-name. 

## Requirements
We will be using the [experimental cognite SDK](https://cognite-sdk-experimental.readthedocs-hosted.com/en/latest/cognite.html#functions). 
To run this example, you first need to install this package:

```
pip install cognite-sdk-experimental
```

We will also be needing at least one asset in your CDF project. 

## Authentication

In order to connect to CDF we will authenticate using Azure AD. There are several ways to do this, all of them outlined in the [Authenticate with Azure AD](https://docs.cognite.com/dev/guides/sdk/python/python_auth_oidc/) documentation. We will in this example use [**client credentials**](https://docs.cognite.com/dev/guides/sdk/python/python_auth_oidc/#authenticate-with-client-secret). There are pros and cons to each authentication method. Here we choose client-credentials due to little additional work needed to get up and running.

This means that we need the following:

1. A `token_client_id` and a `token_client_secret`;
2. a `token_url` (can be inferred from the Azure tenant-ID); and finally,
3. a list of `token_scopes` (can usually be inferred from the CDF cluster). 


## Setup

We start by importing the experimental `CogniteClient` and log in by passing in the required credentials. We use `getpass` to avoid having to paste confidential information directly in the noteobok cell. 

In [ ]:
from getpass import getpass
from cognite.experimental import CogniteClient

project = "<CDF project>"      # Fill in your project here
cdf_cluster  = "<CDF cluster>" # Fill in the cluster your project is running in (for instance 'api'/'westeurope-1')

tenant_id           = "<Tenant ID>"       # Fill in your Azure AD tenant ID here
token_client_id     = "<Token Client ID>" # Fill in your Client ID here
token_client_secret = getpass("Paste your `token_client_secret` in here: ")


client = CogniteClient(
    project = project,
    server  = cdf_cluster,
    client_name = "my first cognite function",
    token_url = token_url,
    token_scopes = token_scopes,
    token_client_id = token_client_id,
    token_client_secret = token_client_secret,
)

If you have no assets in your project, you can create some using the below code-snippet. We will need at least one asset for our function example. 

In [ ]:
# If you have no assets in your CDF project but would like to add some for this example, run this cell.
if len(client.assets.list()) == 0:
    from cognite.client.data_classes import Asset
    
    client.assets.create([Asset(name="my_first_asset"), Asset(name="my_second_asset")])

If we want to verify whether we have logged in successfully, and what capabilites our credentials has, we can call `client.iam.token.inspect()`.

## Defining our function

With an authenticated Cognite Client, we are now ready to define the Python function we would like to deploy as a Cognite Function. 

> Note the distinction between a *python function* (running locally) and a *Cognite Function* (a deployed Python function running directly in CDF). 

The Python function we would like to deploy **must** be named `handle`, and it can take a set of different arguments. The function arguments must be a subset of the following:

1. `client`: By accepting `client` as a function argument, your Cognite Function receives a pre-instantiated `CogniteClient` at call-time. This `client` can be used to interact with CDF from within your function. We will use this to list assets in our example. 

2. `data`: By accepting `data` as a function argument, you can pass additional input data to your function at call-time. We will use this to pass the asset-number we would like the function to access. Note that `data` must be JSON-serializable. 

3. `secrets`: By accepting `secrets` as a function argument, you can get access to confidential values that was attached to the Cognite Function when deploying. We do not use this argument in this example.

4. `function_call_info`. By accepting `function_call_info` we get access to some information pertaining to specific function-execution. We will not use this in this example. 

Below, we define our function-handle. Again, note that currently this is purely a Python function.

In [ ]:
def handle(client, data):
    # We fetch the asset number from the data-dictionary.
    asset_number = data["assetNo"]
    # We use the passed in client to list all assets in our CDF project.
    all_assets = client.assets.list()
    # We finally print the name of the selected asset, and return the name.
    print(all_assets[asset_number].name)
    return {
        "assetName": all_assets[asset_number].name
    }

### Testing our function locally

Before we deploy this Python-function as a Cognite Function, we can verify that it works locally by passing in our previously instantiated CogniteClient and a data-dictionary containing an asset-number. 

In [ ]:
handle(client, {"assetNo" : 1})

## Deploying the function to CDF as a Cognite Function

We now deploy our `handle` function to CDF. We give the function a name and an external ID, and we reference the `handle`-function by using the `function_handle`-argument. When deploying, the function will have status `Queued`, then `Deploying`, and finally `Ready`. 

In [ ]:
fn_name = "my first cognite function"
external_id = "my_first_cognite_function"

function = client.functions.create(
    name = fn_name,
    external_id = external_id,
    function_handle = handle
)

We can run `.update()` and `.status` to get the current status of the function deployment. When it says `Ready` we are ready to call the function deployed in CDF. This usually takes between 3 and 5 minutes. 

In [ ]:
function.update()
function.status

The function is now successfully deployed. We can now call it.

## Calling the Cognite Function

Since we have the `function`-object readily available, we can call the function directly by passing in some data. This creates a `call`-object, which we can use to fetch logs and response from the executed function. Note that the `call`-object itself does not carry the logs and responses. We have to explicitly fetch those.

In [ ]:
call = function.call(data={"assetNo": 0})
call

We view logs and responses:

In [ ]:
call.get_response()

In [ ]:
call.get_logs()